In [32]:
import pandas as pd
from finvizfinance.screener.overview import Overview

def get_filtered_picks(countries=['USA'], limit_per_country=5000):
    all_stocks = []
    
    for country in countries:
        print(f"Fetching candidates for: {country}...")
        
        try:
            foverview = Overview()
            
            # --- STEP 1: Broaden the initial search ---
            filters_dict = {
                'Analyst Recom.': 'Strong Buy (1)',
                'Country': country,
                'Average Volume': 'Over 2M', # Added volume filter for liquidity
                'Market Cap.': '+Small (over $300mln)'
            }
            foverview.set_filter(filters_dict=filters_dict)
            
            # Get results
            df_results = foverview.screener_view()
            
            if not df_results.empty:
                df_results['Source_Country'] = country
                all_stocks.append(df_results)
            else:
                print(f"   No stocks found for {country}.")
                
        except Exception as e:
            print(f"   Error fetching {country}: {e}")

    if not all_stocks:
        print("\nNo stocks found from any country.")
        return pd.DataFrame()

    # Combine all results
    combined_df = pd.concat(all_stocks, ignore_index=True)
    
    # --- STEP 2: The "Option 2" Pandas Filter ---
    cols = combined_df.columns.tolist()
    recom_col = 'Recom' if 'Recom' in cols else 'Analyst Recom'
    
    if recom_col in cols:
        # Convert column to numbers
        combined_df[recom_col] = pd.to_numeric(combined_df[recom_col], errors='coerce')
        
        # Filter: keep anything <= 1.5 (Strong Buys)
        print(f"Filtering out 'Hold' or worse (Rating > 1.5)...")
        combined_df = combined_df[combined_df[recom_col] <= 1.0]
        
        # Sort by best rating
        combined_df = combined_df.sort_values(by=recom_col, ascending=True)

    # Clean up columns
    desired_cols = ['Ticker', 'Company', 'Sector', 'Price', 'Source_Country', recom_col]
    final_cols = [c for c in desired_cols if c in combined_df.columns]
    
    return combined_df[final_cols].head(limit_per_country * len(countries))

# --- Usage ---
top_picks = get_filtered_picks(['USA'])

if not top_picks.empty:
    # Ensure Price is numeric (Crucial for sorting in Data Viewer)
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    print(f"\nSuccess! Found {len(top_picks)} candidates.")
    print("Variable 'top_picks' is ready for the Data Viewer.")

else:
    print("DataFrame is empty.")

Fetching candidates for: USA...
[Info] loading page [###########################---] 10/11 
Success! Found 207 candidates.
Variable 'top_picks' is ready for the Data Viewer.


In [33]:
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    mid_picks = top_picks[(top_picks['Price'] > 5) & (top_picks['Price'] < 30)]

    print(f"\nFound {len(mid_picks)} stocks between $5 and $30.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 110 stocks between $5 and $30.


In [34]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    cheap_picks = top_picks[(top_picks['Price'] < 5)]

    print(f"\nFound {len(cheap_picks)} stocks less than $5")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 28 stocks less than $5


In [35]:
# filter for price range less than $5 a bit gambly
if not top_picks.empty:
    # Ensure the 'Price' column is numeric
    top_picks['Price'] = pd.to_numeric(top_picks['Price'], errors='coerce')

    # CORRECT WAY: Use '&' with parentheses
    Main_TFSA_picks = top_picks[(top_picks['Price'] > 20)]

    print(f"\nFound {len(Main_TFSA_picks)} stocks greater than $20.")
    # print(cheap_picks) # Uncomment if you want to see the list in output
else:
    print("DataFrame is empty.")


Found 98 stocks greater than $20.


In [36]:
"""
import yfinance as yf
import time

# Loop through EVERY symbol in the list (removed the [:3])
for symbol in ticker_list:
    
    # 1. Fetch data
    stock = yf.Ticker(symbol)
    print(f"\n--- Checking {symbol} ---")
    
    try:
        # 2. Get the upgrades/downgrades
        upgrades = stock.upgrades_downgrades
        
        if not upgrades.empty:
            # Filter for 2025 actions using the .index fix
            recent = upgrades[upgrades.index >= '2025-11-01']
            
            if not recent.empty:
                print(recent)
            else:
                print(f"No actions for {symbol} in 2025.")
        else:
            print(f"No analyst history found for {symbol}.")
            
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")

    # 3. CRITICAL: Sleep for 1 second to avoid getting banned
    time.sleep(1)
"""

'\nimport yfinance as yf\nimport time\n\n# Loop through EVERY symbol in the list (removed the [:3])\nfor symbol in ticker_list:\n    \n    # 1. Fetch data\n    stock = yf.Ticker(symbol)\n    print(f"\n--- Checking {symbol} ---")\n    \n    try:\n        # 2. Get the upgrades/downgrades\n        upgrades = stock.upgrades_downgrades\n        \n        if not upgrades.empty:\n            # Filter for 2025 actions using the .index fix\n            recent = upgrades[upgrades.index >= \'2025-11-01\']\n            \n            if not recent.empty:\n                print(recent)\n            else:\n                print(f"No actions for {symbol} in 2025.")\n        else:\n            print(f"No analyst history found for {symbol}.")\n            \n    except Exception as e:\n        print(f"Error fetching {symbol}: {e}")\n\n    # 3. CRITICAL: Sleep for 1 second to avoid getting banned\n    time.sleep(1)\n'